## Acquire process:

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


# acquire
from env import host, user, password
from pydataset import data

In [2]:
# Create helper function to get the necessary connection url.

def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'


In [3]:
def get_telco_churn_data():
    '''
    This function reads in the iris data from the Codeup db
    and returns a pandas DataFrame with all columns.
    '''
    
    sql_query = '''
    SELECT *
    FROM customers
    JOIN contract_types ON customers.contract_type_id = contract_types.contract_type_id
    JOIN payment_types ON customers.payment_type_id = payment_types.payment_type_id
    JOIN internet_service_types ON customers.internet_service_type_id = internet_service_types.internet_service_type_id
    '''
    return pd.read_sql(sql_query, get_connection('telco_churn'))

_________

## Explore the data

In [4]:
#rename data to 'telco'
telco = get_telco_churn_data()

In [ ]:
#look at data
telco.head()

In [ ]:
#this data appears to have no nulls
#data has int and object data types
telco.info()

### data summary:
- 18 object data types
- 8 integer data types
- 1 float data type
- 0 null values


In [ ]:
#descriptive statistics
telco.describe()

_______

### get curious about the data

In [ ]:
#we are trying to determine churn... look into that
telco.churn.value_counts()

#this shows 26.54% of customers churn (1869 out of 7043)

In [ ]:
#find ou how many are senior citizens and how many are not
telco.senior_citizen.value_counts()

#this shows that 16.21% of all customers churn (1142 out of 7043)

In [ ]:
#average tenure, max tenure, min tenure??
telco.tenure.mean(), telco.tenure.max(), telco.tenure.min()

In [ ]:
#average monthly charges, max monthly charges, min monthly charges
telco.monthly_charges.mean(), telco.monthly_charges.max(), telco.monthly_charges.min()

In [ ]:
#group by churn and senior_citizen 
#senior_churn= telco.groupby(['churn','senior_citizen'])

_________

In [ ]:
#gettting value counts for churn as int not obj
(telco.churn == 'Yes').astype(int).value_counts()

In [ ]:
telco.head()

In [ ]:
telco.head()

In [ ]:
#can see that churn is now an int
telco.info()

### Data Prep

In [5]:
#change total charges from an object to a float- put that in PREPARE
telco.total_charges = telco.total_charges.str.replace(' ', '0').astype(float)

In [6]:
##need to convert churn to int
telco_change = telco.replace({'churn': {'No':0, 'Yes':1}}, inplace=True)
## ^can only run line once

In [7]:
#drop columns not needed
telco = telco.drop(['customer_id'], axis=1)

In [8]:
telco.head()

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,online_backup,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,payment_type_id,payment_type,internet_service_type_id,internet_service_type
0,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,Yes,...,2,90.45,5957.90,0,3,Two year,2,Mailed check,1,DSL
1,Male,0,No,No,54,No,No phone service,1,Yes,No,...,4,45.20,2460.55,0,3,Two year,4,Credit card (automatic),1,DSL
2,Female,0,No,No,56,No,No phone service,1,Yes,Yes,...,3,45.05,2560.10,0,3,Two year,3,Bank transfer (automatic),1,DSL
3,Male,0,Yes,Yes,20,No,No phone service,1,Yes,No,...,4,39.40,825.40,0,3,Two year,4,Credit card (automatic),1,DSL
4,Male,0,Yes,No,72,Yes,Yes,1,No,Yes,...,3,85.15,6316.20,0,3,Two year,3,Bank transfer (automatic),1,DSL


In [17]:
#find correlation- I think age correlates the most to churn
telco_correlation = telco.corr()
telco_correlation
#this shows that senior_citizen and monthly_charges have the highest pos corr with churn

,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,payment_type_id,internet_service_type_id
senior_citizen,1.000000,0.016567,-0.032310,-0.142554,-0.093704,0.220173,0.103006,0.150889,-0.142554,-0.093704,-0.032310
tenure,0.016567,1.000000,-0.030359,0.671607,0.340305,0.247900,0.826178,-0.352229,0.671607,0.340305,-0.030359
internet_service_type_id,-0.032310,-0.030359,1.000000,0.099721,0.008124,-0.323260,-0.175755,-0.047291,0.099721,0.008124,1.000000
contract_type_id,-0.142554,0.671607,0.099721,1.000000,0.358913,-0.074195,0.446855,-0.396713,1.000000,0.358913,0.099721
payment_type_id,-0.093704,0.340305,0.008124,0.358913,1.000000,-0.074353,0.222398,-0.262818,0.358913,1.000000,0.008124
monthly_charges,0.220173,0.247900,-0.323260,-0.074195,-0.074353,1.000000,0.651174,0.193356,-0.074195,-0.074353,-0.323260
total_charges,0.103006,0.826178,-0.175755,0.446855,0.222398,0.651174,1.000000,-0.198324,0.446855,0.222398,-0.175755
churn,0.150889,-0.352229,-0.047291,-0.396713,-0.262818,0.193356,-0.198324,1.000000,-0.396713,-0.262818,-0.047291
contract_type_id,-0.142554,0.671607,0.099721,1.000000,0.358913,-0.074195,0.446855,-0.396713,1.000000,0.358913,0.099721
payment_type_id,-0.093704,0.340305,0.008124,0.358913,1.000000,-0.074353,0.222398,-0.262818,0.358913,1.000000,0.008124


In [18]:
#this gives the all the correlation with JUST churn
telco_correlation['churn']

## again, senior_citizen is the highest

senior_citizen              0.150889
tenure                     -0.352229
internet_service_type_id   -0.047291
contract_type_id           -0.396713
payment_type_id            -0.262818
monthly_charges             0.193356
total_charges              -0.198324
churn                       1.000000
contract_type_id           -0.396713
payment_type_id            -0.262818
internet_service_type_id   -0.047291
Name: churn, dtype: float64

_______

### Modeling

#### Find the appropriate model to use
- churn (numerical) and senior_citizen (numerical)
- 2 continuous variables
- 

In [19]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   gender                    7043 non-null   object 
 1   senior_citizen            7043 non-null   int64  
 2   partner                   7043 non-null   object 
 3   dependents                7043 non-null   object 
 4   tenure                    7043 non-null   int64  
 5   phone_service             7043 non-null   object 
 6   multiple_lines            7043 non-null   object 
 7   internet_service_type_id  7043 non-null   int64  
 8   online_security           7043 non-null   object 
 9   online_backup             7043 non-null   object 
 10  device_protection         7043 non-null   object 
 11  tech_support              7043 non-null   object 
 12  streaming_tv              7043 non-null   object 
 13  streaming_movies          7043 non-null   object 
 14  contract

### Hypothosis:

- **$H_{0}$**: there is no relationshp between churn and customer age
- **$H_{a}$**: there is a relationshp between churn and customer age
- **True Positive**: I reject the $H_{0}$, and the network latency has chanced since switching providers
- **True Negative**: I fail to reject the $H_{0}$, and the network latency has not chanced since switching providers
- **False Positive**: I reject the $H_{0}$, but switching provides does not change network latency
- **False Negative**: I fail to reject the $H_{0}$, but switching provides does change network latency

In [ ]:
## crosstab, confusion matrix

In [ ]:
sns.histplot(x='variable', y='value', hue= 'species_name', data=melt)
plt.show()

In [ ]:
sns.barplot(x=train.species_name, y=train.index)

In [ ]:
###Train Split

In [23]:
import prepare
import acquire

In [24]:
train, validate, test = prepare.prep_telco_churn(acquire.get_telco_churn_data())

KeyError: "['customer_id' 'internet_service_id' 'contract_type_id' 'payment_type_id'] not found in axis"

In [ ]:
pd.crosstab(train.survived, train.model5)